In [2]:
import numpy as np
import tensorflow as tf

def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])
    
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i)          for i in range(101, 2 ** NUM_DIGITS)])

NUM_HIDDEN = 100

X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

w_h = init_weights([NUM_DIGITS, NUM_HIDDEN])
w_o = init_weights([NUM_HIDDEN, 4])

def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

predict_op = tf.argmax(py_x, 1)

def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

BATCH_SIZE = 128

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for epoch in range(10000):
        p = np.random.permutation(range(len(trX)))
        trX, trY = trX[p], trY[p]
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})
        if epoch % 100 == 0 :
            print(epoch, np.mean(np.argmax(trY, axis=1) ==
                    sess.run(predict_op, feed_dict={X: trX, Y: trY})))
    numbers = np.arange(1, 101)
    teX = np.transpose(binary_encode(numbers, NUM_DIGITS))
    teY = sess.run(predict_op, feed_dict={X: teX})
    output = np.vectorize(fizz_buzz)(numbers, teY)

    print(output)

(0, 0.5297941495124594)
(100, 0.53412784398699886)
(200, 0.53412784398699886)
(300, 0.53412784398699886)
(400, 0.53412784398699886)
(500, 0.53412784398699886)
(600, 0.53412784398699886)
(700, 0.5449620801733478)
(800, 0.58938244853737809)
(900, 0.61755146262188521)
(1000, 0.68255687973997836)
(1100, 0.7356446370530878)
(1200, 0.73347778981581802)
(1300, 0.74431202600216684)
(1400, 0.76814734561213438)
(1500, 0.81798483206933914)
(1600, 0.81690140845070425)
(1700, 0.85698808234019497)
(1800, 0.86890574214517879)
(1900, 0.88624052004333698)
(2000, 0.89057421451787644)
(2100, 0.90790899241603462)
(2200, 0.92307692307692313)
(2300, 0.92849404117009748)
(2400, 0.90574214517876495)
(2500, 0.94149512459371609)
(2600, 0.94691224268689056)
(2700, 0.94582881906825567)
(2800, 0.94799566630552545)
(2900, 0.95124593716143013)
(3000, 0.95557963163596971)
(3100, 0.95016251354279524)
(3200, 0.9566630552546046)
(3300, 0.95232936078006503)
(3400, 0.94799566630552545)
(3500, 0.96316359696641385)
(3600, 0